<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Specify device
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# Import all necessary libraries.
import sys
import cv2

In [3]:
# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')

sys.path.append(NOMEROFF_NET_DIR)

In [4]:
# Import license plate recognition tools.
from NomeroffNet.YoloV5Detector import Detector
detector = Detector()
detector.load()

yolov5s-2021-12-14.pt: 0.00B [00:00, ?B/s]

Downloaded model path: /mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/Detector/yolov5/yolov5s-2021-12-14.pt


yolov5s-2021-12-14.pt: 14.4MB [00:06, 2.12MB/s]                            
YOLOv5 🚀 v6.0-128-g581dc30 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients


In [5]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft, getCvZoneRGB, convertCvZonesRGBtoBGR, reshapePoints
npPointsCraft = NpPointsCraft()
npPointsCraft.load()

craft_mlt_25k_2020-02-16.pth: 0.00B [00:00, ?B/s]

Downloaded model path: /mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_mlt/craft_mlt_25k_2020-02-16.pth


craft_mlt_25k_2020-02-16.pth: 83.2MB [00:41, 2.00MB/s]                            
craft_refiner_CTW1500_2020-02-16.pth: 0.00B [00:00, ?B/s]

Downloaded model path: /mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth


craft_refiner_CTW1500_2020-02-16.pth: 1.86MB [00:02, 841kB/s]                             


Loading weights from checkpoint (/mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [6]:
from NomeroffNet.OptionsDetector import OptionsDetector

optionsDetector = OptionsDetector()
optionsDetector.load("modelhub://numberplate_options_uacustom")

numberplate_options_2021_12_16_uacustom_pytorch_lightning.ckpt: 0.00B [00:00, ?B/s]

Downloaded model path: /mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/OptionsDetector/numberplate_options_uacustom/numberplate_options_2021_12_16_uacustom_pytorch_lightning.ckpt


numberplate_options_2021_12_16_uacustom_pytorch_lightning.ckpt: 173MB [01:24, 2.05MB/s]                              


NPOptionsNet(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [7]:
from NomeroffNet.TextDetector import TextDetector
from NomeroffNet.TextPostprocessing import textPostprocessing

textDetector = TextDetector.get_static_module("eu")
textDetector.load("latest")

anpr_ocr_eu_2021_09_27_pytorch_lightning.ckpt: 0.00B [00:00, ?B/s]

Downloaded model path: /mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/TextDetector/Eu/anpr_ocr_eu_2021_09_27_pytorch_lightning.ckpt


anpr_ocr_eu_2021_09_27_pytorch_lightning.ckpt: 30.2MB [00:13, 2.20MB/s]                            


NPOcrNet(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [15]:
# Detect numberplate
img_path = 'images/example2.jpeg'
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

targetBoxes = detector.detect_bbox(img)
all_points = npPointsCraft.detect(img, targetBoxes,[5,2,0])

# cut zones
zonesRGB = [getCvZoneRGB(img, reshapePoints(rect, 1)) for rect in all_points]
zones = convertCvZonesRGBtoBGR(zonesRGB)

# predict zones attributes 
region_ids, count_lines, classification_score = optionsDetector.predict(zones, return_acc=True)
regions_confidences, count_lines_confidences = classification_score
region_names = optionsDetector.getRegionLabels(region_ids)
print("[INFO] count lines local:", count_lines, count_lines_confidences)
print("[INFO] region names local:", region_names, regions_confidences)

count_lines, count_lines_confidences = optionsDetector.custom_count_lines_id_to_all_count_lines_with_confidences(
    count_lines, 
    count_lines_confidences)
region_ids, regions_confidences = optionsDetector.custom_regions_id_to_all_regions_with_confidences(
    region_ids, 
    regions_confidences)

region_names = optionsDetector.get_regions_label_global(region_ids)

print("[INFO] region ids global:", region_ids, count_lines)
print("[INFO] count lines global:", count_lines, count_lines_confidences)
print("[INFO] region names global:", region_names, regions_confidences)

# find text with postprocessing by standart
text_arr = textDetector.predict(zones)
text_arr = textPostprocessing(text_arr, region_names)
print("[INFO] predicted numberplate text:", text_arr)

[INFO] count lines local: [1, 1] [[    0.99989  5.5694e-05  5.7711e-05]
 [    0.99996  2.1953e-05  2.2861e-05]]
[INFO] region names local: ['eu', 'eu'] [[ 0.00048235  0.00041441   0.0003879     0.99447   0.0003043  0.00036309  0.00023279   0.0010931    0.002247]
 [   0.025951   0.0051571    0.018703     0.85621    0.014339   0.0054719    0.019554    0.036791    0.017826]]
[INFO] region ids global: [4, 4] [1, 1]
[INFO] count lines global: [1, 1] [[0, 0.99988663, 5.5693956e-05, 5.771109e-05], [0, 0.9999552, 2.195349e-05, 2.2861046e-05]]
[INFO] region names global: ['eu', 'eu'] [[0, 0.00048235312, 0.00041440682, 0.00038789868, 0.99447495, 0.00030429853, 0, 0, 0.00036308586, 0.0002327875, 0.0010931287, 0, 0.0022470213, 0, 0, 0, 0], [0, 0.025951471, 0.0051571243, 0.018703206, 0.85620654, 0.014339294, 0, 0, 0.005471927, 0.019553522, 0.036790986, 0, 0.017825957, 0, 0, 0, 0]]
[INFO] predicted numberplate text: ['RP70012', 'JJF509']


/mnt/data/var/www/nomeroff-net/NomeroffNet/nnmodels/numberplate_options_model.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x1 = functional.softmax(self.fc3_reg(x1))
/mnt/data/var/www/nomeroff-net/NomeroffNet/nnmodels/numberplate_options_model.py:123: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x2 = functional.softmax(self.fc3_line(x2))
